<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0515iwashita_yoshihara_BERT_mlm_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install transformers
    #!pip install transformers==4.5.0
    !pip install fugashi
    #!pip install fugashi==1.1.0
    !pip install ipadic
    #!pip install ipadic==1.0.0

In [ ]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# if device == torch.device('cuda:0'):
#     bert_mlm = bert_mlm.cuda()
# else:
#     bert_mlm = bert_mlm.cpu()

In [ ]:
inp_text = '今日は[MASK]へ行く。'
inp_text = 'ジュースをお願いします'
inp_text = '[MASK]をお願いします'
inp_text = '宇宙ステーションはどこにあるんですか。'
inp_text = '宇宙は重力がありませんから、歩くことができないんです。'
inp_tokens = tokenizer.tokenize(inp_text)
print(inp_tokens)

inp_text = '宇宙は[MASK]がありませんから、歩くことができないんです。'
inp_tokens = tokenizer.tokenize(inp_text)
print(inp_tokens)


In [ ]:
# 文章を符号化し、GPUに配置する。
input_ids = tokenizer.encode(inp_text, return_tensors='pt')
#input_ids = input_ids.device

# BERTに入力し、分類スコアを得る。
# 系列長を揃える必要がないので、単に iput_ids のみを入力します。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [ ]:
# ID 列で '[MASK]' (IDは4) の位置を調べる
mask_position = input_ids[0].tolist().index(4) 

# スコアが最も良いトークンのIDを取り出し、トークンに変換する。
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

# [MASK]を上で求めたトークンで置き換える。
inp_text = inp_text.replace('[MASK]', token_best)

print(inp_text)

In [ ]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    #input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4) 
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

#text = '今日は[MASK]へ行く。'
inp_text = '[MASK]をお願いします'
inp_text = '宇宙[MASK]はどこにあるんですか。'
inp_text = '宇宙は[MASK]がありませんから、歩くことができないんです。'
text_topk, _ = predict_mask_topk(inp_text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

#text = '今日は[MASK][MASK]へ行く。'
inp_text = '[MASK]をお願いし[MASK]'
inp_text = '宇宙[MASK]は[MASK]にあるんですか。'
inp_text = '宇宙は[MASK]がありませんから、歩くことができないんです。'

greedy_prediction(inp_text, tokenizer, bert_mlm)

In [ ]:
# 5-9
inp_text = '今日は[MASK][MASK][MASK][MASK][MASK]'
inp_text = '宇宙[MASK]は[MASK][MASK]あるんですか。'
inp_text = '宇宙は[MASK]がありませんから、歩くことができないんです。'

greedy_prediction(inp_text, tokenizer, bert_mlm)

In [ ]:
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

inp_text = "今日は[MASK][MASK]へ行く。"
inp_text = '宇宙は[MASK]がありませんから、歩くことができないんです。'
text_topk = beam_search(inp_text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
# 5-11
inp_text = '今日は[MASK][MASK][MASK][MASK][MASK]'
inp_text = '宇宙は[MASK]がありませんから、[MASK]ことができないんです。'

text_topk = beam_search(inp_text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
import gzip
import json

data_fname = 'ccap/2022_0205minnichi_data.json.gz'
lines = {}
with gzip.open(data_fname, 'rb') as fgz:
    tmp = json.loads(fgz.read().decode('utf-8'))
    for k in tmp.keys():
        lines[k] = tmp[k]
                    
_max_length, max_length = 0, -1
vocab, freq = ['<EOS>','<SOS>','<UNK>','<PAD>','<MASK>'], {}
for i in range(len(lines)):
    if _max_length < lines[str(i)]['n_token']:
        _max_length = lines[str(i)]['n_token']

    tokens = lines[str(i)]['tokens']
    for token in tokens:
        if not token in vocab:
            vocab.append(token)
            freq[token] = 1
        else:
            freq[token] += 1
    freq = freq
    if max_length == -1:
        max_length = _max_length
    else:
        max_length = max_length
    vocab = vocab


In [ ]:
vocab